In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
import os, json, cv2, random
from numpy import load

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer

In [2]:
#NumpyarrPath = "/content/drive/MyDrive/UnivJobData/augmentedDATA.npz"
JSONpath = "../Dataset/augumentLevel1/augmentedJSON.json"  ##Argument 1
classes = ["Common line","Fuel line 1","Fuel line 2","Fuel line 3", "Fuel line 4"]  ##Argument 2
imgPath = "../Dataset/augumentLevel1/"  ##Argument 3
programName = 'engine_'  ##Argument 4
hasKeypoints = True  ##Argument 5
hasSegmentationAndBBox = True  ##Argument 6
onlyBBox = False  ##Argument 7

if hasKeypoints == True:
    maxNumOfKeypoints = 16  ## give only if your dataset has keypoints   ##Argument 8

with open(JSONpath) as f:
    imgs_anns = json.load(f)

### Without segmentations
def get_dicts(img_dir,imgs_anns,hasKeypoints,hasSegmentationAndBBox,onlyBBox):
    dataset_dicts = []
    for idx, v in enumerate(imgs_anns):
      record = {}
      filename = os.path.join(img_dir, v["image"])
      height, width = cv2.imread(filename).shape[:2]
      record["file_name"] = filename
      record["image_id"] = idx
      record["height"] = height
      record["width"] = width

      #### code for key points start ###
      if "kp-1" in v:
        print("hello")
        kps = v["kp-1"]
        obkps=[]
        countOfKeypoints = len(kps)
        for _, kp in enumerate(kps):
          x_kp = kp["x"] * width/100
          y_kp = kp["y"] * height/100
          v_kp = 1 #look at keypoints, it is not visible but labeled
          obkps.append(x_kp)
          obkps.append(y_kp)
          obkps.append(v_kp)
        
        if countOfKeypoints < maxNumOfKeypoints:
          for kp in range(0,(maxNumOfKeypoints-countOfKeypoints)):
            obkps.append(0)
            obkps.append(0)
            obkps.append(0)
      #else:  ## Remove if your dataset does not have keypoints
      #  obkps=[]
      #  for kp in range(0,maxNumOfKeypoints):
      #    obkps.append(0)
      #    obkps.append(0)
      #    obkps.append(0)
        #### code for key points ends ###
      if "label" in v:
        annos = v["label"]
      objs = []
      noSegmentationPoints = 0
      for _, anno in enumerate(annos):
        if "label" in v:              
          getPoints = np.array(anno["points"])
          if len(getPoints) != 0:
            getPoints[:,0] = getPoints[:,0] * width/100
            getPoints[:,1] = getPoints[:,1] * height/100
            px = getPoints[:,0]
            py = getPoints[:,1]
            poly = (getPoints.flatten()).tolist()
          else:
            noSegmentationPoints = 1

        for ind in range(0,len(classes)):
          if anno["polygonlabels"][0] == classes[ind]:
            catId= ind        
        if ("kp-1" in v) and ("label" in v) and (noSegmentationPoints == 0):
          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [],
              "category_id": catId,
              "keypoints": obkps,
          }
        elif ("label" in v) and (noSegmentationPoints == 0):
          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [poly],
              "category_id": catId,
              "keypoints": [],
          }
        elif ("kp-1" in v) and (noSegmentationPoints == 1):
          obj = {
              "keypoints": obkps,
              "segmentation": [],
              "bbox": [],
              "bbox_mode": BoxMode.XYXY_ABS,
              "category_id": []
          }
        elif ("kp-1" in v) and ("label" in v) and (noSegmentationPoints == 1):
          obj = {
              "keypoints": obkps,
              "segmentation": [],
              "bbox": BoxMode.XYXY_ABS,
              "bbox_mode": BoxMode.XYXY_ABS,
              "category_id": catId
          }
        objs.append(obj)
      record["annotations"] = objs
      dataset_dicts.append(record)
    return dataset_dicts


In [3]:
#def get_dicts(Images,imgs_anns,hasKeypoints,hasSegmentationAndBBox,onlyBBox):
def get_dicts(img_dir,imgs_anns,hasKeypoints,hasSegmentationAndBBox,onlyBBox):
    dataset_dicts = []
    for idx, v in enumerate(imgs_anns):
      record = {}
      filename = os.path.join(img_dir, v["image"])
      height, width = cv2.imread(filename).shape[:2]
      record["file_name"] = filename
      record["image_id"] = idx
      record["height"] = height
      record["width"] = width

      #### code for key points start ###
      if ("kp-1" in v) and (hasKeypoints == True):
        kps = v["kp-1"]
        obkps=[]
        countOfKeypoints = len(kps)
        for _, kp in enumerate(kps):
          x_kp = kp["x"] * width/100
          y_kp = kp["y"] * height/100
          v_kp = 1 #look at keypoints, it is not visible but labeled
          obkps.append(x_kp)
          obkps.append(y_kp)
          obkps.append(v_kp)
        
        if countOfKeypoints < maxNumOfKeypoints:
          for kp in range(0,(maxNumOfKeypoints-countOfKeypoints)):
            obkps.append(0)
            obkps.append(0)
            obkps.append(0)
      elif hasKeypoints == True:  ## Remove if your dataset does not have keypoints
        obkps=[]
        for kp in range(0,maxNumOfKeypoints):
          obkps.append(0)
          obkps.append(0)
          obkps.append(0)
        #### code for key points ends ###
      if "label" in v and (hasSegmentationAndBBox == True):
        annos = v["label"]
      objs = []
      noSegmentationPoints = 0
      for _, anno in enumerate(annos):
        if "label" in v:              
          getPoints = np.array(anno["points"])
          if len(getPoints) != 0:
            getPoints[:,0] = getPoints[:,0] * width/100
            getPoints[:,1] = getPoints[:,1] * height/100
            px = getPoints[:,0]
            py = getPoints[:,1]
            poly = (getPoints.flatten()).tolist()
          else:
            noSegmentationPoints = 1   ## If any image does not contains segmentations

        for ind in range(0,len(classes)):
          if anno["polygonlabels"][0] == classes[ind]:
            catId= ind        
        if ("kp-1" in v) and ("label" in v) and (noSegmentationPoints == 0):
          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [poly],
              "category_id": catId,
              "keypoints": obkps,
          }
        elif ("label" in v) and (noSegmentationPoints == 0) and (onlyBBox == False) and (hasSegmentationAndBBox == True):
          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [poly],
              "category_id": catId,
              "keypoints": [],
          }
        elif ("label" in v) and (noSegmentationPoints == 0) and onlyBBox == True:
          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [],
              "category_id": catId,
              "keypoints": [],
          }
        elif ("kp-1" in v) and (noSegmentationPoints == 1):
          obj = {
              "keypoints": obkps,
              "segmentation": [],
              "bbox": [],
              "bbox_mode": BoxMode.XYXY_ABS,
              "category_id": []
          }
        elif ("kp-1" in v) and ("label" in v) and (noSegmentationPoints == 1):
          obj = {
              "keypoints": obkps,
              "segmentation": [],
              "bbox": BoxMode.XYXY_ABS,
              "bbox_mode": BoxMode.XYXY_ABS,
              "category_id": catId
          }
        objs.append(obj)
      record["annotations"] = objs
      dataset_dicts.append(record)
    return dataset_dicts


In [4]:
for d in ["train"]:
    DatasetCatalog.register(programName + d, lambda d=d: get_dicts(imgPath,imgs_anns,hasKeypoints,hasSegmentationAndBBox,onlyBBox))
    MetadataCatalog.get(programName + d).set(thing_classes=classes)
    MetadataCatalog.get(programName + d).set(keypoint_names = ["kp1","kp2","kp3","kp4","kp5","kp6","kp7","kp8","kp9","kp10","kp11","kp12","kp13","kp14","kp15","kp16"])
    MetadataCatalog.get(programName + d).set(keypoint_flip_map = [])
engine_metadata = MetadataCatalog.get(programName+"_train")

In [5]:
for d in random.sample(get_dicts(imgPath,imgs_anns,hasKeypoints,hasSegmentationAndBBox,onlyBBox), 2):
#for d in dataset_dicts:
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=engine_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2.imshow("output",out.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

cfg.DATASETS.TRAIN = (programName+"train",)
cfg.MODEL.MASK_ON = True
#cfg.MODEL.KEYPOINT_ON = True
cfg.DATASETS.TEST = ()  
cfg.DATALOADER.NUM_WORKERS = 2

cfg.SOLVER.IMS_PER_BATCH = 2
#cfg.MODEL.BACKBONE.FREEZE_AT = 2
cfg.SOLVER.BASE_LR = 0.0004
cfg.SOLVER.WEIGHT_DECAY = 0.0001
cfg.SOLVER.GAMMA = 0.008
cfg.SOLVER.MOMENTUM = 0.95
cfg.SOLVER.MAX_ITER = 1500
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 100   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
#cfg.MODEL.SEM_SEG_HEAD.NORM = "GN"
#cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 4
#cfg.MODEL.ROI_KEYPOINT_HEAD.NORMALIZE_LOSS_BY_VISIBLE_KEYPOINTS = True
cfg.MODEL.ROI_KEYPOINT_HEAD.LOSS_WEIGHT = 1.0
#cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = maxNumOfKeypoints
#cfg.TEST.KEYPOINT_OKS_SIGMAS = np.ones((maxNumOfKeypoints, 1), dtype=float).tolist()

cfg.OUTPUT_DIR = "../Dataset/output"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# set the testing threshold for this model
#cfg.OUTPUT_DIR = "../Dataset/outputv2"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

In [ ]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9
predictor = DefaultPredictor(cfg)